In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
import numpy as np
# dataframe functions
from pyspark.sql import functions as fn
from __future__ import division
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml import Pipeline
import pandas as pd
from pyspark.ml.feature import RegexTokenizer
# we obtain the stop words from a website
import requests
stop_words = requests.get('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words').text.split()
stop_words[0:10]
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder
import seaborn
from pyspark.ml import feature
from pyspark.sql import types

In [2]:
stop_words.append('https')
stop_words.append('t')

In [ ]:
##turning polarity into stretched 0-5 scrores

#new = int((old + 1) * 2.999999999999)

In [ ]:
#we can create intensity scores from polarity but the issue then is how do we make a log of that,
#next best thing is to create more classses and then look at statistical differences from those!

In [ ]:
#scenario 1: turn all scores into positive and negative

In [3]:
fullDFP=pd.read_csv('fulldatasetT.csv',dtype={'polarity': float})

In [4]:
fullDFP=fullDFP.loc[:,['id','text','place','polarity']]

In [5]:
fullDFP['score']=0

In [6]:
#data_df.loc[data_df["mean radius"] > 12.0, "mean radius"] = 0
fullDFP.loc[fullDFP["polarity"] > 0.0, 'score'] = 1

In [7]:
fullDFP

,id,text,place,polarity,score
0,1.29E+18,Yoga Instructor🧘‍♀️ @bare_table rocks it Frida...,"Kentucky, USA",0.1767,1
1,1.29E+18,Coz a nice stroll in the city is like a ghost ...,"Sydney, New South Wales",0.3333,1
2,1.29E+18,NEW! #Facemasks in my Society6 #Shop 👉 https:/...,Martinique,0.1705,1
3,1.29E+18,"I’m not saying I’m bored, but I have invented ...","Boise, ID",-0.1356,0
4,1.29E+18,Fighting Stigma: NPHET discuss regional lockdo...,"Mascouche, Québec",0.0000,0
...,...,...,...,...,...
151575,1.37E+18,Call us. We’ll keep you safe. Always wear a ma...,"Boca Raton, FL",0.5000,1
151576,1.37E+18,What is #themoment you knew the pandemic was c...,"Manhattan, NY",-0.1519,0
151577,1.37E+18,I’ve had just about every post-vaccine side ef...,"Sapulpa, OK",0.0958,1
151578,1.37E+18,Updated: Closure on #SouthernStateParkway EB a...,"North Valley Stream, NY",0.0000,0


In [8]:
from pyspark.sql.types import *

mySchema = StructType([StructField("id", StringType(), True)\

                       ,StructField("text", StringType(), True)\

                       ,StructField("place", StringType(), True)\

                       ,StructField("polarity", FloatType(), True)\
                       
                       ,StructField("score", IntegerType(), True)])

In [9]:
fullDf=spark.createDataFrame(fullDFP,schema=mySchema)

In [10]:
fullDf.where(fn.col('score')==1).show(10)

+--------+--------------------+--------------------+--------+-----+
|      id|                text|               place|polarity|score|
+--------+--------------------+--------------------+--------+-----+
|1.29E+18|Yoga Instructor🧘...|       Kentucky, USA|  0.1767|    1|
|1.29E+18|Coz a nice stroll...|Sydney, New South...|  0.3333|    1|
|1.29E+18|NEW! #Facemasks i...|          Martinique|  0.1705|    1|
|1.29E+18|Fighting Stigma: ...|   Mascouche, Québec|  0.1238|    1|
|1.29E+18|BON APPETITE

Eva...|     Bal Harbour, FL|  0.0111|    1|
|1.29E+18|We are probably m...|      Tennessee, USA|  0.0606|    1|
|1.29E+18|Great #summer nig...|        Pasadena, CA|     0.6|    1|
|1.29E+18|Waited all day to...|         Phoenix, AZ|  0.2964|    1|
|1.29E+18|Another new month...|  Navi Mumbai, India|  0.1811|    1|
|1.29E+18|@ChelseaFC

let's...|       Ogun, Nigeria|     0.5|    1|
+--------+--------------------+--------------------+--------+-----+
only showing top 10 rows



In [14]:
imdb_reviews_df = spark.read.parquet('imdb_reviews_preprocessed.parquet')
sentiments_df=spark.read.parquet('sentiments.parquet')

In [15]:
tokenizer = RegexTokenizer().setGaps(False)\
  .setPattern("\\p{L}+")\
  .setInputCol("text")\
  .setOutputCol("words")
review_words_df = tokenizer.transform(fullDf)


In [16]:
review_words_df.show()

+--------+--------------------+--------------------+--------+-----+--------------------+
|      id|                text|               place|polarity|score|               words|
+--------+--------------------+--------------------+--------+-----+--------------------+
|1.29E+18|Yoga Instructor🧘...|       Kentucky, USA|  0.1767|    1|[yoga, instructor...|
|1.29E+18|Coz a nice stroll...|Sydney, New South...|  0.3333|    1|[coz, a, nice, st...|
|1.29E+18|NEW! #Facemasks i...|          Martinique|  0.1705|    1|[new, facemasks, ...|
|1.29E+18|I’m not saying I’...|           Boise, ID| -0.1356|    0|[i, m, not, sayin...|
|1.29E+18|Fighting Stigma: ...|   Mascouche, Québec|     0.0|    0|[fighting, stigma...|
|1.29E+18|Fighting Stigma: ...|   Mascouche, Québec| -0.3125|    0|[fighting, stigma...|
|1.29E+18|Fighting Stigma: ...|   Mascouche, Québec|  0.1238|    1|[fighting, stigma...|
|1.29E+18|Them COVID nights...|Vandenberg Villag...|     0.0|    0|[them, covid, nig...|
|1.29E+18|Shoot after 

In [17]:
tweet_words_sentiment_df = review_words_df.\
    select('id', fn.explode('words').alias('word')).\
    join(sentiments_df, 'word')
tweet_words_sentiment_df.show(5)

+----------+--------+---------+
|      word|      id|sentiment|
+----------+--------+---------+
|confidence|1.29E+18|        1|
|    strong|1.29E+18|        1|
|      nice|1.29E+18|        1|
|      like|1.29E+18|        1|
|protection|1.29E+18|        1|
+----------+--------+---------+
only showing top 5 rows



In [18]:
simple_sentiment_prediction_df = tweet_words_sentiment_df.\
    groupBy('id').\
    agg(fn.avg('sentiment').alias('avg_sentiment')).\
    withColumn('predicted', fn.when(fn.col('avg_sentiment') > 0, 1.0).otherwise(0.))
simple_sentiment_prediction_df.show(5)

+--------+-------------------+---------+
|      id|      avg_sentiment|predicted|
+--------+-------------------+---------+
|1.31E+18|0.13998100664767332|      1.0|
|1.32E+18| 0.1479426096372496|      1.0|
|1.30E+18| 0.3137938922566625|      1.0|
|1.36E+18|0.12607260726072608|      1.0|
|1.37E+18|0.22395476353666896|      1.0|
+--------+-------------------+---------+
only showing top 5 rows



In [19]:
fullDf.\
    join(simple_sentiment_prediction_df, 'id').\
    select(fn.expr('float(score = predicted)').alias('correct')).\
    select(fn.avg('correct')).\
    show()

+------------------+
|      avg(correct)|
+------------------+
|0.5745594648264578|
+------------------+



In [20]:
sw_filter = StopWordsRemover()\
  .setStopWords(stop_words)\
  .setCaseSensitive(False)\
  .setInputCol("words")\
  .setOutputCol("filtered")

# we will remove words that appear in 5 docs or less
cv = CountVectorizer(minTF=1., minDF=5., vocabSize=2**17)\
  .setInputCol("filtered")\
  .setOutputCol("tf")


# we now create a pipelined transformer
cv_pipeline = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(fullDf)
# now we can make the transformation between the raw text and the counts
cv_pipeline.transform(fullDf)


DataFrame[id: string, text: string, place: string, polarity: float, score: int, words: array<string>, filtered: array<string>, tf: vector]

In [21]:
len(cv_pipeline.stages[-1].vocabulary)

31661

In [22]:

idf = IDF().\
    setInputCol('tf').\
    setOutputCol('tfidf')
lr = LogisticRegression().\
    setLabelCol('score').\
    setFeaturesCol('tfidf').\
    setRegParam(0.0).\
    setMaxIter(100).\
    setElasticNetParam(0.)
idf_pipeline = Pipeline(stages=[cv_pipeline, idf]).fit(fullDf)


In [23]:
training_df, validation_df, testing_df = fullDf.randomSplit([0.6, 0.3, 0.1], seed=0)

In [24]:
lr_pipeline = Pipeline(stages=[idf_pipeline, lr]).fit(training_df)

In [25]:
lr_pipeline.transform(validation_df).\
    select(fn.expr('float(prediction = score)').alias('correct')).\
    select(fn.avg('correct')).show()

+------------------+
|      avg(correct)|
+------------------+
|0.7767307395228742|
+------------------+



In [26]:
vocabulary = idf_pipeline.stages[0].stages[-1].vocabulary
weights = lr_pipeline.stages[-1].coefficients.toArray()
coeffs_df = pd.DataFrame({'word': vocabulary, 'weight': weights})

In [27]:
coeffs_df.sort_values('weight').head(10)

,word,weight
5241,devastating,-24.353693
27348,hapiness,-22.054723
5057,awful,-21.530582
28815,shutterstock,-20.263581
23952,crooked,-19.802672
28689,burdens,-19.573643
20776,forecasting,-18.849654
17144,turtles,-18.692263
10741,terrifying,-18.420199
14858,fearful,-18.305814


In [28]:
lambda_par = 0.02
alpha_par = 0.3
en_lr = LogisticRegression().\
        setLabelCol('score').\
        setFeaturesCol('tfidf').\
        setRegParam(lambda_par).\
        setMaxIter(100).\
        setElasticNetParam(alpha_par)
en_lr_estimator = Pipeline(
    stages=[tokenizer, sw_filter, cv, idf, en_lr])
en_lr_pipeline = en_lr_estimator.fit(training_df)

In [29]:
en_lr_pipeline.transform(validation_df).select(fn.avg(fn.expr('float(prediction = score)'))).show()

+--------------------------------+
|avg(float((prediction = score)))|
+--------------------------------+
|              0.8710965947961954|
+--------------------------------+



In [30]:
en_weights = en_lr_pipeline.stages[-1].coefficients.toArray()
en_coeffs_df = pd.DataFrame({'word': en_lr_pipeline.stages[2].vocabulary, 'weight': en_weights})
en_coeffs_df.sort_values('weight').head(15)

,word,weight
1317,worst,-0.218300
443,bad,-0.201350
1664,hate,-0.182759
496,crazy,-0.178315
2035,stupid,-0.170983
1134,sick,-0.170353
985,cold,-0.165276
441,soda,-0.162481
6824,goldkilos,-0.151609
996,sad,-0.127827


In [31]:
en_coeffs_df.sort_values('weight', ascending=False).head(15)

,word,weight
5,new,0.648823
28,happy,0.604511
32,good,0.561014
25,love,0.544115
39,great,0.530618
88,best,0.527535
29,safe,0.512404
106,beautiful,0.476345
92,free,0.417571
146,available,0.409601


In [32]:
en_coeffs_df.query('weight == 0.0').shape
en_coeffs_df.query('weight == 0.0').shape[0]/en_coeffs_df.shape[0]

0.9807051637978901

In [33]:
en_coeffs_df.query('weight == 0.0').head(15)

,word,weight
0,covid,0.0
4,coronavirus,0.0
10,home,0.0
11,m,0.0
13,quarantine,0.0
14,health,0.0
17,stigma,0.0
18,mask,0.0
20,just,0.0
21,like,0.0


In [34]:
grid = ParamGridBuilder().\
    addGrid(en_lr.regParam, [0., 0.01, 0.02]).\
    addGrid(en_lr.elasticNetParam, [0., 0.2, 0.4]).\
    build()
all_models = []
for j in range(len(grid)):
    print("Fitting model {}".format(j+1))
    model = en_lr_estimator.fit(training_df, grid[j])
    all_models.append(model)

Fitting model 1
Fitting model 2
Fitting model 3
Fitting model 4
Fitting model 5
Fitting model 6
Fitting model 7
Fitting model 8
Fitting model 9


In [36]:
# estimate the accuracy of each of them:
accuracies = [m.\
    transform(validation_df).\
    select(fn.avg(fn.expr('float(score = prediction)')).alias('accuracy')).\
    first().\
    accuracy for m in all_models]
accuracies

[0.8185951051574604,
 0.8185951051574604,
 0.8185951051574604,
 0.8496899344558957,
 0.899499040010593,
 0.8864122878644097,
 0.8480568490278728,
 0.8866109063624126,
 0.8571270937699997]

In [37]:
best_model_idx = np.argmax(accuracies)

In [38]:
grid[best_model_idx]

{Param(parent='LogisticRegression_5a34156cb995', name='regParam', doc='regularization parameter (>= 0).'): 0.01,
 Param(parent='LogisticRegression_5a34156cb995', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.2}

In [39]:
best_model = all_models[best_model_idx]
# estimate generalization performance
best_model.\
    transform(testing_df).\
    select(fn.avg(fn.expr('float(score = prediction)')).alias('accuracy')).\
    show()

+------------------+
|          accuracy|
+------------------+
|0.9022833453971179|
+------------------+



## EEC corpus as input for the Best LR model

In [61]:
#Loading Equity Evaluation Corpus to use as testing data
eec =pd.read_csv('Equity-Evaluation-Corpus.csv')

In [89]:
eec

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed
...,...,...,...,...,...,...,...,...
8635,2018-En-mystery-12020,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,funny
8636,2018-En-mystery-14529,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,hilarious
8637,2018-En-mystery-16746,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,amazing
8638,2018-En-mystery-00046,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,wonderful


In [62]:
#Dropping other columns
eec_test = eec.drop(["ID", "Template", "Person", "Gender", "Race", "Emotion", "Emotion word"], axis= 1)

In [77]:
eec_test_spark= spark.createDataFrame(eec)


TypeError: field Race: Can not merge type <class 'pyspark.sql.types.StringType'> and <class 'pyspark.sql.types.DoubleType'>

In [76]:
#Converting the pandas df to a spark df
eec_test_spark= spark.createDataFrame(eec_test)

#renaming the input column as "Sentence" from "text"
eec_test_spark = eec_test_spark.withColumnRenamed("sentence","text")

In [85]:
predictions_lr_t = best_model.transform(training_df)
predictions_lr_t.show()

+--------+--------------------+--------------------+--------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|      id|                text|               place|polarity|label|               words|            filtered|                  tf|               tfidf|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|1.29E+18|!! sigalert !! a ...|          Corona, CA|  0.1429|    1|[sigalert, a, cra...|[sigalert, crash,...|(21612,[16,47,117...|(21612,[16,47,117...|[-0.8939347319445...|[0.29029850478073...|       1.0|
|1.29E+18|!! sigalert !! a ...|          Corona, CA|  0.1429|    1|[sigalert, a, cra...|[sigalert, crash,...|(21612,[16,47,117...|(21612,[16,47,117...|[-0.8939347319445...|[0.29029

In [68]:
#Tranforming the EEC corpus in the Best model
predictions_lr = best_model.transform(eec_test_spark)
predictions.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|               words|            filtered|                  tf|               tfidf|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| Alonzo feels angry.|[alonzo, feels, a...|[alonzo, feels, a...|(31661,[564,5514]...|(31661,[564,5514]...|[8.65560021335638...|[0.43278001066781...|       1.0|
|Alonzo feels furi...|[alonzo, feels, f...|[alonzo, feels, f...|(31661,[564,22084...|(31661,[564,22084...|[8.65560021335638...|[0.43278001066781...|       1.0|
|Alonzo feels irri...|[alonzo, feels, i...|[alonzo, feels, i...|(31661,[564,20624...|(31661,[564,20624...|[8.65560021335638...|[0.43278001066781...|       1.0|
|Alonzo feels enra...|[alonzo, feels, e.

In [71]:
lr_raw_predictions_df = predictions_lr.select("text", "rawPrediction","prediction")

In [82]:
lr_raw_predictions_df = lr_raw_predictions_df.toPandas()

### Random Forest

In [54]:
# Preload packages
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
# dataframe functions
from pyspark.sql import functions as fn
import os
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
import requests
stop_words = requests.get('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words').text.split()
stop_words[0:10]
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import IDF
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pandas as pd
from pyspark.sql.types import *

from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [40]:
###import dataset

fullDFP=pd.read_csv('fulldatasetT.csv',dtype={'polarity': float})
fullDFP=fullDFP.loc[:,['id','text','place','polarity']]
fullDFP['score']=0
fullDFP.loc[fullDFP["polarity"] > 0.0, 'score'] = 1

mySchema = StructType([StructField("id", StringType(), True)\

                       ,StructField("text", StringType(), True)\

                       ,StructField("place", StringType(), True)\

                       ,StructField("polarity", FloatType(), True)\
                       
                       ,StructField("score", IntegerType(), True)])
fullDf=spark.createDataFrame(fullDFP,schema=mySchema)

In [45]:
sentiments_df=spark.read.parquet('sentiments.parquet')

In [41]:
fullDf=fullDf.withColumnRenamed('score','label')

In [42]:
fullDf.describe().show()

+-------+-----------+--------------------+--------------------+--------+------------------+
|summary|         id|                text|               place|polarity|             label|
+-------+-----------+--------------------+--------------------+--------+------------------+
|  count|     151580|              151580|              151580|  151580|            151580|
|   mean|        NaN|                 NaN|                 NaN|     NaN| 0.574548093416018|
| stddev|        NaN|                 NaN|                 NaN|     NaN|0.4944129796127013|
|    min|   1.27E+18| https://t.co/Xjr...|'s-Hertogenbosch,...|    -1.0|                 0|
|    max|Rome, Lazio|🪂[Everything you...|Эльбрусский район...|     NaN|                 1|
+-------+-----------+--------------------+--------------------+--------+------------------+



In [43]:
training_df, validation_df, testing_df = fullDf.randomSplit([0.6, 0.3, 0.1])

In [52]:
tokenizer = RegexTokenizer().setGaps(False)\
  .setPattern("\\p{L}+")\
  .setInputCol("text")\
  .setOutputCol("words")
review_words_df = tokenizer.transform(fullDf)
tweet_words_sentiment_df = review_words_df.\
    select('id', fn.explode('words').alias('word')).\
    join(sentiments_df, 'word')
tweet_words_sentiment_df.show(5)

sw_filter = StopWordsRemover()\
  .setStopWords(stop_words)\
  .setCaseSensitive(False)\
  .setInputCol("words")\
  .setOutputCol("filtered")

# we will remove words that appear in 5 docs or less
cv = CountVectorizer(minTF=1., minDF=5., vocabSize=2**17)\
  .setInputCol("filtered")\
  .setOutputCol("tf")

# we now create a pipelined transformer
cv_pipeline = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(fullDf)

idf = IDF().\
    setInputCol('tf').\
    setOutputCol('tfidf')
lr = LogisticRegression().\
    setLabelCol('label').\
    setFeaturesCol('tfidf').\
    setRegParam(0.0).\
    setMaxIter(100).\
    setElasticNetParam(0.)
idf_pipeline = Pipeline(stages=[cv_pipeline, idf]).fit(fullDf)

+----------+--------+---------+
|      word|      id|sentiment|
+----------+--------+---------+
|confidence|1.29E+18|        1|
|    strong|1.29E+18|        1|
|      nice|1.29E+18|        1|
|      like|1.29E+18|        1|
|protection|1.29E+18|        1|
+----------+--------+---------+
only showing top 5 rows



In [55]:
rf = RandomForestClassifier().setLabelCol('label').\
    setFeaturesCol('tfidf')
rf_pipeline = Pipeline(stages=[idf_pipeline, rf]).fit(training_df)

In [56]:
bce = BinaryClassificationEvaluator()
bce.evaluate(rf_pipeline.transform(validation_df))

0.705945387153658

In [58]:
predictions = rf_pipeline.transform(testing_df)
predictions.filter(predictions['prediction'] == 0) \
    .select("text",'probability',"label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+------------------------------+-----+----------+
|                          text|                   probability|label|prediction|
+------------------------------+------------------------------+-----+----------+
|Would you like to help a bi...|[0.5558719509419924,0.44412...|    0|       0.0|
|Would you like to help a bi...|[0.5558719509419924,0.44412...|    0|       0.0|
|Would you like to help a bi...|[0.5558719509419924,0.44412...|    0|       0.0|
|Would you like to help a bi...|[0.5558719509419924,0.44412...|    0|       0.0|
|Would you like to help a bi...|[0.5558719509419924,0.44412...|    0|       0.0|
|Would you like to help a bi...|[0.5558719509419924,0.44412...|    0|       0.0|
|Would you like to help a bi...|[0.5558719509419924,0.44412...|    0|       0.0|
|Would you like to help a bi...|[0.5510694896786703,0.44893...|    0|       0.0|
|Would you like to help a bi...|[0.5510694896786703,0.44893...|    0|       0.0|
|Would you like to help a bi

In [117]:
predictions_rf = rf_pipeline.transform(eec_test_spark)

In [79]:
predictions_rf.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|               words|            filtered|                  tf|               tfidf|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| Alonzo feels angry.|[alonzo, feels, a...|[alonzo, feels, a...|(31661,[564,5514]...|(31661,[564,5514]...|[8.65560021335638...|[0.43278001066781...|       1.0|
|Alonzo feels furi...|[alonzo, feels, f...|[alonzo, feels, f...|(31661,[564,22084...|(31661,[564,22084...|[8.65560021335638...|[0.43278001066781...|       1.0|
|Alonzo feels irri...|[alonzo, feels, i...|[alonzo, feels, i...|(31661,[564,20624...|(31661,[564,20624...|[8.65560021335638...|[0.43278001066781...|       1.0|
|Alonzo feels enra...|[alonzo, feels, e.

In [110]:
rf_raw_predictions_df = predictions_rf.select("text", "rawPrediction","prediction")

In [81]:
rf_raw_predictions_df = rf_raw_predictions_df.toPandas()

In [92]:
rf_raw_predictions_df["id"] = rf_raw_predictions_df.index +1

In [93]:
lr_raw_predictions_df["id"] = lr_raw_predictions_df.index +1

In [97]:
new_df=lr_raw_predictions_df.merge(rf_raw_predictions_df, on= "id")

In [107]:
new_df

,text_x,rawPrediction_x,prediction_x,id,text_y,rawPrediction_y,prediction_y
0,Alonzo feels angry.,"[1.7587616811561602, -1.7587616811561602]",0.0,1,Alonzo feels angry.,"[8.655600213356383, 11.344399786643615]",1.0
1,Alonzo feels furious.,"[1.1883767559140832, -1.1883767559140832]",0.0,2,Alonzo feels furious.,"[8.655600213356383, 11.344399786643615]",1.0
2,Alonzo feels irritated.,"[1.1883767559140832, -1.1883767559140832]",0.0,3,Alonzo feels irritated.,"[8.655600213356383, 11.344399786643615]",1.0
3,Alonzo feels enraged.,"[1.1883767559140832, -1.1883767559140832]",0.0,4,Alonzo feels enraged.,"[8.655600213356383, 11.344399786643615]",1.0
4,Alonzo feels annoyed.,"[1.1883767559140832, -1.1883767559140832]",0.0,5,Alonzo feels annoyed.,"[8.655600213356383, 11.344399786643615]",1.0
...,...,...,...,...,...,...,...
8635,The conversation with my mom was funny.,"[-0.26419630122863724, 0.26419630122863724]",1.0,8636,The conversation with my mom was funny.,"[8.325650019702103, 11.674349980297892]",1.0
8636,The conversation with my mom was hilarious.,"[0.4793126425382578, -0.4793126425382578]",0.0,8637,The conversation with my mom was hilarious.,"[8.655600213356383, 11.344399786643615]",1.0
8637,The conversation with my mom was amazing.,"[-1.2568624618888387, 1.2568624618888387]",1.0,8638,The conversation with my mom was amazing.,"[8.234367928179505, 11.765632071820491]",1.0
8638,The conversation with my mom was wonderful.,"[-1.1733344977347269, 1.1733344977347269]",1.0,8639,The conversation with my mom was wonderful.,"[8.655600213356383, 11.344399786643615]",1.0


In [99]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import pandas as pd
from sklearn import preprocessing

x = new_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
new_df = pd.new_df(x_scaled)

In [122]:
new_df["new_val1"] = 0

In [123]:
for i in range(len(new_df)):
    new_df.iloc[i,7]= new_df["rawPrediction_x"][i][0]

In [132]:
new_df["new_val2"] = 0

In [133]:
for i in range(len(new_df)):
    new_df.iloc[i,8]= new_df["rawPrediction_y"][i][0]

In [134]:
new_df

,text_x,rawPrediction_x,prediction_x,id,text_y,rawPrediction_y,prediction_y,new_val1,new_val2
0,Alonzo feels angry.,"[1.7587616811561602, -1.7587616811561602]",0.0,1,Alonzo feels angry.,"[8.655600213356383, 11.344399786643615]",1.0,1.758762,8.655600
1,Alonzo feels furious.,"[1.1883767559140832, -1.1883767559140832]",0.0,2,Alonzo feels furious.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600
2,Alonzo feels irritated.,"[1.1883767559140832, -1.1883767559140832]",0.0,3,Alonzo feels irritated.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600
3,Alonzo feels enraged.,"[1.1883767559140832, -1.1883767559140832]",0.0,4,Alonzo feels enraged.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600
4,Alonzo feels annoyed.,"[1.1883767559140832, -1.1883767559140832]",0.0,5,Alonzo feels annoyed.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600
...,...,...,...,...,...,...,...,...,...
8635,The conversation with my mom was funny.,"[-0.26419630122863724, 0.26419630122863724]",1.0,8636,The conversation with my mom was funny.,"[8.325650019702103, 11.674349980297892]",1.0,-0.264196,8.325650
8636,The conversation with my mom was hilarious.,"[0.4793126425382578, -0.4793126425382578]",0.0,8637,The conversation with my mom was hilarious.,"[8.655600213356383, 11.344399786643615]",1.0,0.479313,8.655600
8637,The conversation with my mom was amazing.,"[-1.2568624618888387, 1.2568624618888387]",1.0,8638,The conversation with my mom was amazing.,"[8.234367928179505, 11.765632071820491]",1.0,-1.256862,8.234368
8638,The conversation with my mom was wonderful.,"[-1.1733344977347269, 1.1733344977347269]",1.0,8639,The conversation with my mom was wonderful.,"[8.655600213356383, 11.344399786643615]",1.0,-1.173334,8.655600


In [135]:
from sklearn.preprocessing import MinMaxScaler

In [136]:
scaler = MinMaxScaler()

In [141]:
new_df_to_be = new_df.loc[:, ["new_val1","new_val2"]]

In [142]:
new_df_to_be

,new_val1,new_val2
0,1.758762,8.655600
1,1.188377,8.655600
2,1.188377,8.655600
3,1.188377,8.655600
4,1.188377,8.655600
...,...,...
8635,-0.264196,8.325650
8636,0.479313,8.655600
8637,-1.256862,8.234368
8638,-1.173334,8.655600


In [145]:
df_scaled = pd.DataFrame(scaler.fit_transform(new_df_to_be),columns=new_df_to_be.columns )

In [146]:
df_scaled

,new_val1,new_val2
0,0.524358,1.000000
1,0.443084,1.000000
2,0.443084,1.000000
3,0.443084,1.000000
4,0.443084,1.000000
...,...,...
8635,0.236106,0.226850
8636,0.342049,1.000000
8637,0.094661,0.012955
8638,0.106563,1.000000


In [147]:
df_scaled["id"] = df_scaled.index +1

In [148]:
df_scaled

,new_val1,new_val2,id
0,0.524358,1.000000,1
1,0.443084,1.000000,2
2,0.443084,1.000000,3
3,0.443084,1.000000,4
4,0.443084,1.000000,5
...,...,...,...
8635,0.236106,0.226850,8636
8636,0.342049,1.000000,8637
8637,0.094661,0.012955,8638
8638,0.106563,1.000000,8639


In [149]:
final_df=new_df.merge(df_scaled, on= "id")

In [150]:
final_df

,text_x,rawPrediction_x,prediction_x,id,text_y,rawPrediction_y,prediction_y,new_val1_x,new_val2_x,new_val1_y,new_val2_y
0,Alonzo feels angry.,"[1.7587616811561602, -1.7587616811561602]",0.0,1,Alonzo feels angry.,"[8.655600213356383, 11.344399786643615]",1.0,1.758762,8.655600,0.524358,1.000000
1,Alonzo feels furious.,"[1.1883767559140832, -1.1883767559140832]",0.0,2,Alonzo feels furious.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600,0.443084,1.000000
2,Alonzo feels irritated.,"[1.1883767559140832, -1.1883767559140832]",0.0,3,Alonzo feels irritated.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600,0.443084,1.000000
3,Alonzo feels enraged.,"[1.1883767559140832, -1.1883767559140832]",0.0,4,Alonzo feels enraged.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600,0.443084,1.000000
4,Alonzo feels annoyed.,"[1.1883767559140832, -1.1883767559140832]",0.0,5,Alonzo feels annoyed.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600,0.443084,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
8635,The conversation with my mom was funny.,"[-0.26419630122863724, 0.26419630122863724]",1.0,8636,The conversation with my mom was funny.,"[8.325650019702103, 11.674349980297892]",1.0,-0.264196,8.325650,0.236106,0.226850
8636,The conversation with my mom was hilarious.,"[0.4793126425382578, -0.4793126425382578]",0.0,8637,The conversation with my mom was hilarious.,"[8.655600213356383, 11.344399786643615]",1.0,0.479313,8.655600,0.342049,1.000000
8637,The conversation with my mom was amazing.,"[-1.2568624618888387, 1.2568624618888387]",1.0,8638,The conversation with my mom was amazing.,"[8.234367928179505, 11.765632071820491]",1.0,-1.256862,8.234368,0.094661,0.012955
8638,The conversation with my mom was wonderful.,"[-1.1733344977347269, 1.1733344977347269]",1.0,8639,The conversation with my mom was wonderful.,"[8.655600213356383, 11.344399786643615]",1.0,-1.173334,8.655600,0.106563,1.000000


In [151]:
eec["id"] = eec.index +1

In [152]:
final_df=final_df.merge(eec, on= "id")

In [153]:
final_df

,text_x,rawPrediction_x,prediction_x,id,text_y,rawPrediction_y,prediction_y,new_val1_x,new_val2_x,new_val1_y,new_val2_y,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,Alonzo feels angry.,"[1.7587616811561602, -1.7587616811561602]",0.0,1,Alonzo feels angry.,"[8.655600213356383, 11.344399786643615]",1.0,1.758762,8.655600,0.524358,1.000000,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,Alonzo feels furious.,"[1.1883767559140832, -1.1883767559140832]",0.0,2,Alonzo feels furious.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600,0.443084,1.000000,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,Alonzo feels irritated.,"[1.1883767559140832, -1.1883767559140832]",0.0,3,Alonzo feels irritated.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600,0.443084,1.000000,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,Alonzo feels enraged.,"[1.1883767559140832, -1.1883767559140832]",0.0,4,Alonzo feels enraged.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600,0.443084,1.000000,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,Alonzo feels annoyed.,"[1.1883767559140832, -1.1883767559140832]",0.0,5,Alonzo feels annoyed.,"[8.655600213356383, 11.344399786643615]",1.0,1.188377,8.655600,0.443084,1.000000,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8635,The conversation with my mom was funny.,"[-0.26419630122863724, 0.26419630122863724]",1.0,8636,The conversation with my mom was funny.,"[8.325650019702103, 11.674349980297892]",1.0,-0.264196,8.325650,0.236106,0.226850,2018-En-mystery-12020,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,funny
8636,The conversation with my mom was hilarious.,"[0.4793126425382578, -0.4793126425382578]",0.0,8637,The conversation with my mom was hilarious.,"[8.655600213356383, 11.344399786643615]",1.0,0.479313,8.655600,0.342049,1.000000,2018-En-mystery-14529,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,hilarious
8637,The conversation with my mom was amazing.,"[-1.2568624618888387, 1.2568624618888387]",1.0,8638,The conversation with my mom was amazing.,"[8.234367928179505, 11.765632071820491]",1.0,-1.256862,8.234368,0.094661,0.012955,2018-En-mystery-16746,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,amazing
8638,The conversation with my mom was wonderful.,"[-1.1733344977347269, 1.1733344977347269]",1.0,8639,The conversation with my mom was wonderful.,"[8.655600213356383, 11.344399786643615]",1.0,-1.173334,8.655600,0.106563,1.000000,2018-En-mystery-00046,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,NaN,joy,wonderful


In [156]:
eec.isnull().sum()

ID                 0
Sentence           0
Template           0
Person             0
Gender             0
Race            2880
Emotion          240
Emotion word     240
id                 0
dtype: int64